In [90]:
import pandapower as pp
import pandapower.networks
import numpy as np
import pandas as pd

In [269]:
#if network == 'IEEE-14' then
net = pp.networks.case14()
#else if network == 'IEEE-30' then
#net = pandapower.networks.case30()
#else if network == 'IEEE-57' then
#net = pandapower.networks.case57()
#else if network == 'IEEE-118' then
#net = pandapower.networks.case118()

In [356]:
# if user selects data generation strategy 1
pp.rundcpp(net)
# state vector x_base
x_base = net.res_bus.va_degree.to_numpy()/180*np.pi
# number of discrete time steps
T = 10000 # user argument
# Dx ~ N(0,Cov_Mat)
mean = np.zeros(x_base.shape)
cov = np.eye(x_base.shape[0])*0.001
delta_x_mat = np.transpose(np.random.multivariate_normal(mean, cov**2, size=T))
x_base_mat = np.repeat(x_base.reshape((x_base.shape[0],-1)), T, axis=1)
x_t_mat = x_base_mat + delta_x_mat
x_t_mat[0,:] = 0

In [340]:
# P_from_node_i_to_node_j
# Pij = (1/bij)*(x[i]-x[j])
# H[line_id,i] = 1/bij
# H[line_id,j] = -1/bij
A_real = np.real(net._ppc['internal']['Bbus'].A)
#H = np.zeros((net.line.shape[0]+net.trafo.shape[0], net.bus.shape[0]))
H = np.zeros((2*(net.line.shape[0]+net.trafo.shape[0]), net.bus.shape[0]))
for i in range(0, net.line.shape[0]):
    # from flows
    H[2*i, net.line.from_bus.values[i]] = -A_real[net.line.from_bus.values[i],net.line.to_bus.values[i]]
    H[2*i, net.line.to_bus.values[i]] = A_real[net.line.from_bus.values[i],net.line.to_bus.values[i]]
    # to flows
    H[2*i+1, net.line.to_bus.values[i]] = -A_real[net.line.to_bus.values[i],net.line.from_bus.values[i]]    
    H[2*i+1, net.line.from_bus.values[i]] = A_real[net.line.to_bus.values[i],net.line.from_bus.values[i]]
    
#for i in range(net.line.shape[0]+1, net.line.shape[0]+net.trafo.shape[0]):
for i in range(net.line.shape[0], net.line.shape[0]+net.trafo.shape[0]):
    # from flows
    H[2*i, net.trafo.hv_bus.values[i-net.line.shape[0]]] = -A_real[net.trafo.hv_bus.values[i-net.line.shape[0]],net.trafo.lv_bus.values[i-net.line.shape[0]]]
    H[2*i, net.trafo.lv_bus.values[i-net.line.shape[0]]] = A_real[net.trafo.hv_bus.values[i-net.line.shape[0]],net.trafo.lv_bus.values[i-net.line.shape[0]]]
    # to flows
    H[2*i+1, net.trafo.lv_bus.values[i-net.line.shape[0]]] = -A_real[net.trafo.lv_bus.values[i-net.line.shape[0]],net.trafo.hv_bus.values[i-net.line.shape[0]]]
    H[2*i+1, net.trafo.hv_bus.values[i-net.line.shape[0]]] = A_real[net.trafo.lv_bus.values[i-net.line.shape[0]],net.trafo.hv_bus.values[i-net.line.shape[0]]]
    
#H = np.vstack((H,np.eye(net.bus.shape[0])))

In [341]:
H.shape

(40, 14)

In [344]:
np.linalg.matrix_rank(H)

13

In [345]:
#z_t = H @ x_t + noise
mean = np.zeros(H.shape[0])
cov = np.eye(H.shape[0])*0.01
noise_mat = np.transpose(np.random.multivariate_normal(mean, cov, size=T))
z_t_mat = np.matmul(H, x_t_mat) + noise_mat
z_t_mat.shape

(40, 10000)

In [167]:
# target with matching pursuit 
from sklearn.linear_model import OrthogonalMatchingPursuit

I = [i for i in range(0,H.shape[1])]
I_fixed = [0,4] # 0 always there, but "1,2,3" could have been "2,4,9"
I_free = [i for i in I if I.index(i) not in I_fixed]

H_s = H[:,I_free]
H_s_transpose = np.transpose(H_s)
B_s = H_s @ np.linalg.inv(H_s_transpose @ H_s) @ H_s_transpose - np.eye(H_s.shape[0])
c = np.zeros((H.shape[1],))
for i in I_fixed[1:]:
    c[i] = 0.1
y = B_s @ H[:,I_fixed] @ c[I_fixed]

for n in range(1,H.shape[0]+1):
    print(n)
    # Bs*a=y, solve for a where cardinality(a) = n_nonzero_coefs
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n, normalize=False)
    omp.fit(B_s, y)
    a = omp.coef_
    # a = H*c => c = (H^T * H)^(-1) * H^T * a
    resulting_c = np.linalg.inv(np.transpose(H[:,1:]) @ H[:,1:]) @ np.transpose(H[:,1:]) @ a
    resulting_c = np.hstack((0,resulting_c))
    print((c[I_fixed]-resulting_c[I_fixed]).reshape(-1,1))
    if np.abs(np.max(c[I_fixed]-resulting_c[I_fixed])) < 1e-4:
        break

1
[[0.       ]
 [0.0773338]]
2
[[0.        ]
 [0.05504013]]
3
[[0.        ]
 [0.02459395]]
4
[[0.00000000e+00]
 [4.57966998e-16]]


In [7]:
y.shape

(20,)

In [164]:
# Targeted least effort based on norm-1
# min norm-1(a) with respect to a and c, where a is in z+a and c is in x+c
# subject to:
# a = H*c -> B_s*a = y
# a[k] = 0.1 where this is the initial desire of the adversary i.e. add 0.1 to the k-th measurement
# all entries of a and c should be within some limits
# -0.1 <= a[i] <= 0.1 for all i going from 0 to #meas-1
# -0.1 <= c[i] <= 0.1 for all i going from 0 to #nodes-1

# input params: H, z, k as integer between 0 and H.shape[0]-1, delta_a, a_lower_bound, a_upper_bound, 
# c_lower_bound, c_upper_bound

from gurobipy import *

delta_a = 0.1
k = 4
a_lower_bound, a_upper_bound, c_lower_bound, c_upper_bound = -100, 100, -100, 100

# new material for targerted with norm 1
I = [i for i in range(0,H.shape[1])]
I_fixed = [0,k] # 0 always there, but "1,2,3" could have been "2,4,9"
# I_fixed = [0, k]
I_free = [i for i in I if I.index(i) not in I_fixed]

# new material for targerted with norm 1
H_s = H[:,I_free]
H_s_transpose = np.transpose(H_s)
B_s = H_s @ np.linalg.inv(H_s_transpose @ H_s) @ H_s_transpose - np.eye(H_s.shape[0])

m = Model('targerted least effort with norm 1')

# define decision variables: a and c vectors

a  = [m.addVar(name='a%d' % i) for i in range(H.shape[0])]
c  = [m.addVar(name='c%d' % i) for i in range(H.shape[1])]

a_positive = [m.addVar(lb=0.0,name='a_pos%d' % i) for i in range(H.shape[0])]
a_negative = [m.addVar(ub=0.0,name='a_neg%d' % i) for i in range(H.shape[0])]

m.update()

# constraint: a[i] = a_positive[i] - a_negative[i]
# not a constraint: |a[i]| = a_positive[i] + a_negative[i]
for i in range(0,H.shape[0]):
    m.addConstr(a[i] == a_positive[i] + a_negative[i])

# 1st constraint: c[k] = delta
m.addConstr(c[0] == 0)
m.addConstr(c[k] == delta_a)

# secure sensor
# m.addConstr(a[1] == 0)

# 2nd constraint: B_s*a = y (replacing a = H*c)
c_aux = [c[i] for i in I_fixed]
for i in range(0,H.shape[0]):
    #m.addConstr(a[i] == np.matmul(H,c)[i])
    m.addConstr(np.matmul(B_s,a)[i] == (B_s @ H[:,I_fixed] @ c_aux)[i])
    
# 3rd constraint: ... <= a[i] <= ...
for i in range(0,H.shape[0]):
    m.addConstr(a[i] <= a_upper_bound)
    m.addConstr(a[i] >= a_lower_bound)
    
# 4th constraint: ... <= c[i] <= ...    
for i in range(0,H.shape[1]):
    m.addConstr(c[i] <= c_upper_bound)
    m.addConstr(c[i] >= c_lower_bound)

#m.setObjective(np.linalg.norm(a,ord=1), GRB.MINIMIZE)
# |a[i]| = a_positive[i] - a_negative[i]
m.setObjective(sum(z[0]-z[1] for z in zip(a_positive, a_negative)), GRB.MINIMIZE)

m.update()

m.optimize()

#least_effort_a_norm_1 = m.getVars()[0:H.shape[0]]
targeted_least_effort_a_norm_1 = [v.x for v in m.getVars()[:H.shape[0]]]
# a for a single point in time
targeted_least_effort_a_norm_1 = np.asarray(targeted_least_effort_a_norm_1)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 110 rows, 74 columns and 422 nonzeros
Model fingerprint: 0x779edeee
Coefficient statistics:
  Matrix range     [9e-04, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e+02]
Presolve removed 104 rows and 59 columns
Presolve time: 0.01s
Presolved: 6 rows, 15 columns, 90 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.2304000e-02   3.227991e-03   0.000000e+00      0s
       1    2.8221000e-02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.822100000e-02


In [165]:
# Targeted least effort based on big-M, where M is a hyper-parameter
# min sum(y) with respect to a, c, and y, where a is in z+a and c is in x+c
# subject to:
# a = H*c -> B_s*a = y
# a[k] = 0.1 where this is the initial desire of the adversary i.e. add 0.1 to the k-th measurement
# all entries of a and c should be within some limits
# -0.1 <= a[i] <= 0.1 for all i going from 0 to #meas-1
# -0.1 <= c[i] <= 0.1 for all i going from 0 to #nodes-1
# y[i] where i going from 0 to #meas-1 and y[i] is integer
# -y[i]*M <= a[i] <= y[i]*M
# if y[i] = 0, then 0 <= a[i] <= 0
# if y[i] = 1, then -M <= a[i] <= M

# input params: H, z, M, k as integer between 0 and H.shape[0]-1, delta_a, a_lower_bound, a_upper_bound, 
# c_lower_bound, c_upper_bound

from gurobipy import *

delta_a = 0.1
k = 4
a_lower_bound, a_upper_bound, c_lower_bound, c_upper_bound = -100, 100, -100, 100

M = 10**4

m = Model('targeted least effort with big-M')

# new material for targerted with norm 1
I = [i for i in range(0,H.shape[1])]
I_fixed = [0,k] # 0 always there, but "1,2,3" could have been "2,4,9"
# I_fixed = [0, k]
I_free = [i for i in I if I.index(i) not in I_fixed]

# new material for targerted with norm 1
H_s = H[:,I_free]
H_s_transpose = np.transpose(H_s)
B_s = H_s @ np.linalg.inv(H_s_transpose @ H_s) @ H_s_transpose - np.eye(H_s.shape[0])

# define decision variables: a and c vectors

a = [m.addVar(name='a%d' % i) for i in range(H.shape[0])]
c = [m.addVar(name='c%d' % i) for i in range(H.shape[1])]
y = [m.addVar(vtype=gurobipy.GRB.BINARY, name='y%d' % i) for i in range(H.shape[0])]

m.update()

# -y[i]*M <= a[i] <= y[i]*M
for i in range(0,H.shape[0]):
    m.addConstr(a[i] <= y[i]*M)
    m.addConstr(a[i] >= -y[i]*M)

# 1st constraint: c[k] = delta
m.addConstr(c[0] == 0)
m.addConstr(c[k] == delta_a)


# secure sensor
# m.addConstr(a[1] == 0)

# 2nd constraint: B_s*a = y (replacing a = H*c)
c_aux = [c[i] for i in I_fixed]
for i in range(0,H.shape[0]):
    #m.addConstr(a[i] == np.matmul(H,c)[i])
    m.addConstr(np.matmul(B_s,a)[i] == (B_s @ H[:,I_fixed] @ c_aux)[i])
    
# 3rd constraint: ... <= a[i] <= ...
for i in range(0,H.shape[0]):
    m.addConstr(a[i] <= a_upper_bound)
    m.addConstr(a[i] >= a_lower_bound)
    
# 4th constraint: ... <= c[i] <= ...    
for i in range(0,H.shape[1]):
    m.addConstr(c[i] <= c_upper_bound)
    m.addConstr(c[i] >= c_lower_bound)

#m.setObjective(np.linalg.norm(a,ord=1), GRB.MINIMIZE)
# |a[i]| = a_positive[i] - a_negative[i]
m.setObjective(sum(y), GRB.MINIMIZE)

m.update()

m.optimize()

targeted_least_effort_a_big_M = [v.x for v in m.getVars()[:H.shape[0]]]
# a for a single point in time
targeted_least_effort_a_big_M = np.asarray(targeted_least_effort_a_big_M)

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 130 rows, 54 columns and 442 nonzeros
Model fingerprint: 0xfe6dab40
Variable types: 34 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [9e-04, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+02]
Presolve removed 111 rows and 25 columns
Presolve time: 0.00s
Presolved: 19 rows, 29 columns, 57 nonzeros
Variable types: 13 continuous, 16 integer (16 binary)
Found heuristic solution: objective 2.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 4 (of 4 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%


In [266]:
from sklearn.decomposition import FastICA

z = z_t_mat[:,0:20] # observe a time window of measurements and perturb the last time step

# length of time window should be great or equal to number of measurements, so let's go for 2 times number of measurements

iteration = []
for i in range(1,z.shape[0]+1):
    for j in ['logcosh','exp','cube']:
        print(i,j)
        ica = FastICA(n_components=i, random_state=0, whiten='unit-variance', fun=j)
        y = ica.fit_transform(np.transpose(z)) # fit_transform takes as argument (n_samples, n_features)
        G = ica.mixing_
        deviation = np.max(np.abs(z - (G @ np.transpose(y)) - ica.mean_.reshape((-1,1))))
        iteration.append((i,j,deviation))
best_n_components, best_fun, best_deviation = sorted(iteration, key = lambda t: t[2])[0]

if best_deviation > 1e-4:
    print('infeasible')

best_n_components = 4
ica = FastICA(n_components=best_n_components, random_state=0, whiten='unit-variance', fun=best_fun)
y = ica.fit_transform(np.transpose(z)) # fit_transform takes as argument (n_samples, n_components)
G = ica.mixing_ # (n_features, n_components), it represents H*A in z = H*A*y and x = A*y

delta_y = np.zeros(y.shape[1])
sigma_2 = 0.000001
mean = np.zeros(delta_y.shape[0])
cov = np.eye(delta_y.shape[0])*sigma_2
delta_y = np.random.multivariate_normal(mean, cov**2)
a = G @ (y[-1,:] + delta_y)

1 logcosh
1 exp
1 cube
2 logcosh
2 exp
2 cube
3 logcosh
3 exp
3 cube
4 logcosh
4 exp
4 cube
5 logcosh
5 exp
5 cube


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


6 logcosh
6 exp
6 cube
7 logcosh


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


7 exp
7 cube
8 logcosh


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


8 exp
8 cube
9 logcosh
9 exp
9 cube
10 logcosh
10 exp
10 cube
11 logcosh
11 exp
11 cube
12 logcosh
12 exp
12 cube
13 logcosh
13 exp


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


13 cube
14 logcosh
14 exp
14 cube
15 logcosh
15 exp
15 cube
16 logcosh
16 exp
16 cube
17 logcosh
17 exp
17 cube
18 logcosh
18 exp
18 cube
19 logcosh
19 exp
19 cube
20 logcosh
20 exp
20 cube


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\decomposition\_fastica.py:118: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


In [387]:
# the following code segments for Brownian motion and Ornstein-Uhlenbeck process is from:
# https://towardsdatascience.com/stochastic-processes-simulation-brownian-motion-the-basics-c1d71585d9f9
# and
# https://towardsdatascience.com/stochastic-processes-simulation-the-ornstein-uhlenbeck-process-e8bff820f3

from dataclasses import dataclass

@dataclass
class OUParams:
    alpha: float  # mean reversion parameter
    gamma: float  # asymptotic mean
    beta: float  # Brownian motion scale (standard deviation)

from typing import Optional

import numpy as np

def get_dW(T: int, random_state: Optional[int] = None) -> np.ndarray:
    """
    Sample T times from a normal distribution,
    to simulate discrete increments (dW) of a Brownian Motion.
    Optional random_state to reproduce results.
    """
    np.random.seed(random_state)
    return np.random.normal(0.0, 1.0, T)


def get_W(T: int, random_state: Optional[int] = None) -> np.ndarray:
    """
    Simulate a Brownian motion discretely samplet at unit time increments.
    Returns the cumulative sum
    """
    dW = get_dW(T, random_state)
    # cumulative sum and then make the first index 0.
    dW_cs = dW.cumsum()
    return np.insert(dW_cs, 0, 0)[:-1]

def get_OU_process(
    T: int,
    OU_params: OUParams,
    X_0: Optional[float] = None,
    random_state: Optional[int] = None,
) -> np.ndarray:
    """
    - T is the sample size.
    - Ou_params is an instance of OUParams dataclass.
    - X_0 the initial value for the process, if None, then X_0 is taken
        to be gamma (the asymptotic mean).
    Returns a 1D array.
    """
    t = np.arange(T, dtype=float) # float to avoid np.exp overflow
    exp_alpha_t = np.exp(-OU_params.alpha * t)
    dW = get_dW(T, random_state)
    integral_W = _get_integal_W(t, dW, OU_params)
    _X_0 = _select_X_0(X_0, OU_params)
    return (
        _X_0 * exp_alpha_t
        + OU_params.gamma * (1 - exp_alpha_t)
        + OU_params.beta * exp_alpha_t * integral_W
    )


def _select_X_0(X_0_in: Optional[float], OU_params: OUParams) -> float:
    """Returns X_0 input if not none, else gamma (the long term mean)."""
    if X_0_in is not None:
        return X_0_in
    return OU_params.gamma


def _get_integal_W(
    t: np.ndarray, dW: np.ndarray, OU_params: OUParams
) -> np.ndarray:
    """Integral with respect to Brownian Motion (W), ∫...dW."""
    exp_alpha_s = np.exp(OU_params.alpha * t)
    integral_W = np.cumsum(exp_alpha_s * dW)
    return np.insert(integral_W, 0, 0)[:-1]

In [388]:
OU_proc = []
for i in range(0,len(net.load)):
    alpha_rv = np.abs(np.random.normal(0.01, 0.01))
    beta_rv = np.abs(np.random.normal(0.005, 0.01))
    OU_params = OUParams(alpha=alpha_rv, gamma=0.0, beta=beta_rv)
    OU_proc.append(get_OU_process(100, OU_params))

In [389]:
from pandas import Series
p_mw_init = net.load.p_mw.values.copy()
OU_proc = np.asarray(OU_proc)
x_t_mat = []
for t in range(0,100):
    delta_load = p_mw_init * (1+OU_proc[:,t])
    net.load.p_mw = Series(delta_load)
    pp.rundcpp(net)
    x_t_mat.append(net.res_bus.va_degree.to_numpy()/180*np.pi)
x_t_mat = np.transpose(np.asarray(x_t_mat))

In [390]:
T = 5000
Z = z_t_mat[:,0:T]
z_mean = (1/(Z.shape[1]-1))*np.sum(Z,axis=1)
list_outer = []
for i in range(0,T):
    list_outer.append(np.outer(Z[:,i] - z_mean,Z[:,i] - z_mean))
sigma_z = sum(list_outer)/(T-1)
lamdas, eigenvectors = np.linalg.eig(sigma_z)

p = Z.shape[0]/Z.shape[1] # the ratio of measurements over number of snapshots

#s = []
#threshold = (1 + np.sqrt(p))**2
#for i in range(0,lamdas.shape[0]):
#    if lamdas[i] > threshold:
#        s.append(i)

mu = []
for i in range(0,lamdas.shape[0]):
    mu.append((lamdas[i]+1-p+np.sqrt((lamdas[i]+1-p)**2-4*lamdas[i]))/2 - 1)

omegas = []
for i in range(0,len(mu)):
    omegas.append((1-p/mu[i]**2)/(1+p/mu[i]))
    
# scenario 1a
#np.random.choice(len(s),1)
i = np.random.choice(lamdas.shape[0],1)[0]
tau = 0.3 # an input parameter
c_i = np.sqrt(tau/((1e-3)**2*omegas[i]/mu[i])) # 1e-3 is the noise_factor
a = c_i * eigenvectors[:,i]

# scenario 1b (use the largest eigenvalue)
tau = 0.3 # an input parameter
i = np.argmax(lamdas)
c_i = np.sqrt(tau/((1e-3)**2*omegas[i]/mu[i])) # 1e-3 is the noise_factor
a = c_i * eigenvectors[:,i]

# scenario 2
tau = 0.3 # an input parameter
c_i = []
for i in range(0,lamdas.shape[0]):
    c_i.append(np.sqrt(tau/((1e-3)**2*lamdas.shape[0]*omegas[i]/mu[i]))) # 1e-3 is the noise_factor
c_i = np.asarray(c_i)
a = eigenvectors @ c_i

In [379]:
# norm-2 squared of delta_thetas could probabilistically become above tau
# np.linalg.norm(delta_theta, ord=2)**2
# where delta_theta is x_normal - x_a